In [ ]:
!git clone https://github.com/svc-develop-team/so-vits-svc -b 4.1-Stable

In [ ]:
!cd ./so-vits-svc && pip install --upgrade pip setuptools && pip install -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118

## dataset download

In [ ]:
sovits_data_dir="/home/ec2-user/SageMaker/tts/so-vits-svc/"  
RAW_DIR=sovits_data_dir + "raw/"
RESULTS_DIR=sovits_data_dir + "results/"
FILELISTS_DIR=sovits_data_dir + "filelists/"
CONFIGS_DIR=sovits_data_dir + "configs/"
LOGS_DIR=sovits_data_dir + "logs/44k/"

#!mkdir -p ${RAW_DIR}"/speaker0"
#!mkdir -p ${RAW_DIR}"/speaker1"
#
#!aws s3 cp "s3://sagemaker-us-west-2-687912291502/video/raw/.3aece165-a9b8-475e-9ae2-447520f47cf2.mp3"  {RAW_DIR}"speaker0/0001.wav"
#!aws s3 cp "s3://sagemaker-us-west-2-687912291502/video/raw/.4f69647e-23d0-4efe-9dec-dc695224ee56.mp3"  {RAW_DIR}"speaker1/0001.wav"

## model download

In [ ]:
# encoder model
#!aws s3 cp s3://sagemaker-us-west-2-687912291502/models/checkpoint_best_legacy_500.pt {sovits_data_dir}/pretrain/
sovits_data_dir="/home/ec2-user/SageMaker/tts/so-vits-svc/" 
!wget -P {sovits_data_dir}/pretrain/ https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -O checkpoint_best_legacy_500.pt

In [ ]:
# pretrain model
download_pretrained_model=True
D_0_URL = "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_D_320000.pth" #@param ["https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_D_320000.pth", "https://huggingface.co/1asbgdh/sovits4.0-volemb-vec768/resolve/main/clean_D_320000.pth", "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/vol_emb/clean_D_320000.pth"] {allow-input: true}
G_0_URL = "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_G_320000.pth" #@param ["https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_G_320000.pth", "https://huggingface.co/1asbgdh/sovits4.0-volemb-vec768/resolve/main/clean_G_320000.pth", "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/vol_emb/clean_G_320000.pth"] {allow-input: true}

#download_pretrained_diffusion_model
download_pretrained_diffusion_model=True
diff_model_URL = "https://huggingface.co/datasets/ms903/Diff-SVC-refactor-pre-trained-model/resolve/main/fix_pitch_add_vctk_600k/model_0.pt" #@param {type:"string"}


if download_pretrained_model:
    !curl -L {D_0_URL} -o {sovits_data_dir}/logs/44k/D_0.pth
    !curl -L {G_0_URL} -o {sovits_data_dir}/logs/44k/G_0.pth


if download_pretrained_diffusion_model:
    !mkdir -p logs/44k/diffusion
    !curl -L {diff_model_URL} -o {sovits_data_dir}/logs/44k/diffusion/model_0.pt


## preprocess

In [ ]:
!ffmpeg -i {sovits_data_dir}/dataset_raw/speaker0/Brigida.wav -f segment -segment_time 60 -c copy out%03d.wav

In [3]:
!cd ./so-vits-svc/ && rm -rf ./dataset_raw/.ipynb_checkpoints
!cd ./so-vits-svc/ && python resample.py
#
##param ["vec768l12", "vec256l9", "hubertsoft", "whisper-ppg", "whisper-ppg-large"]
speech_encoder="vec768l12" 
use_vol_aug = False
vol_aug = "--vol_aug" if use_vol_aug else ""
#!cd ./so-vits-svc/ && python preprocess_flist_config.py --speech_encoder={speech_encoder} {vol_aug}
!cd ./so-vits-svc/ && python preprocess_flist_config.py  --speech_encoder={speech_encoder}

CPU count: 8
./dataset_raw/speaker0
resampling: ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00m 0:00:01
100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 7530.17it/s]
2024-03-26 02:35:18.870 | INFO     | __main__:<module>:74 - Writing ./filelists/train.txt
100%|█████████████████████████████████████████| 3/3 [00:00<00:00, 136770.78it/s]
2024-03-26 02:35:18.871 | INFO     | __main__:<module>:80 - Writing ./filelists/val.txt
100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 96420.78it/s]
2024-03-26 02:35:18.876 | INFO     | __main__:<module>:115 - Writing to configs/config.json
2024-03-26 02:35:18.876 | INFO     | __main__:<module>:118 - Writing to configs/diffusion.yaml


In [ ]:
# for tts model train：
!cd ./so-vits-svc/ && python preprocess_hubert_f0.py --f0_predictor dio

In [4]:
# for diffusion model train only:
!cd ./so-vits-svc/ && python preprocess_hubert_f0.py --f0_predictor dio --use_diff

vec768l12
2024-03-26 02:35:32.639 | INFO     | __main__:<module>:152 - Using device: cuda:0
2024-03-26 02:35:32.639 | INFO     | __main__:<module>:153 - Using SpeechEncoder: vec768l12
2024-03-26 02:35:32.639 | INFO     | __main__:<module>:154 - Using extractor: dio
2024-03-26 02:35:32.639 | INFO     | __main__:<module>:155 - Using diff Mode: True
use_diff
Loading Mel Extractor...
Loaded Mel Extractor.
  0%|                                                     | 0/1 [00:00<?, ?it/s]2024-03-26 02:35:35.024 | INFO     | __mp_main__:process_batch:107 - Loading speech encoder for content...
2024-03-26 02:35:35.056 | INFO     | __mp_main__:process_batch:113 - Rank 1 uses device cuda:0
2024-03-26 02:35:35.392852: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 02:35:35.392909: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to

## train

In [ ]:
config_path = sovits_data_dir + "./configs/config.json"
!cd ./so-vits-svc && python train.py -c {config_path} -m 44k

In [ ]:
#!cd ./so-vits-svc && python cluster/train_cluster.py --gpu

In [ ]:
#!cd /content/so-vits-svc && python train_index.py -c configs/config.json

In [ ]:
### optinal: train diffusion model
%cd ./so-vits-svc

import os
if not os.path.exists("./pretrain/nsf_hifigan/model"):
  !curl -L https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-v1/nsf_hifigan_20221211.zip -o nsf_hifigan_20221211.zip
  !unzip nsf_hifigan_20221211.zip
  !rm -rf pretrain/nsf_hifigan
  !mv -v nsf_hifigan pretrain


!python train_diff.py -c configs/diffusion.yaml

In [20]:
## prediction
!cd ./so-vits-svc && python inference_main.py -m "logs/44k/G_10000.pth" -c "configs/config.json" -n "../../validate003.mp3" -t 0 -s "speaker0" --f0_predictor dio 

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
emb_g.weight is not in the checkpoint,please check your checkpoint.If you're using pretrain model,just ignore this warning.
load 
2024-03-28 09:33:34.412764: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 09:33:34.412808: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 09:33:34.417666: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLA

In [22]:
!cd ./so-vits-svc && python inference_main.py --loudness_envelope_adjustment 0.7 --shallow_diffusion --diffusion_model_path "logs/44k/diffusion/model_100000.pt" --diffusion_config_path "logs/44k/diffusion/config.yaml" -m "logs/44k/G_10000.pth" -c "configs/config.json" -n "../../validate003.wav" -t 0 -s "speaker0" --f0_predictor dio
                                              #--use_spk_mix \
                                              #--only_diffusion \

 [Loading] logs/44k/diffusion/model_100000.pt
Loaded diffusion model, sampler is dpm-solver++, speedup: 10 
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
emb_g.weight is not in the checkpoint,please check your checkpoint.If you're using pretrain model,just ignore this warning.
load 
2024-03-28 09:34:53.706036: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 09:34:53.706095: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28

## webui startup for test

In [ ]:
!cd ./so-vits-svc && python webUI.py